### Importing libraries

In [1]:
import pymongo
from pymongo import MongoClient
import json
import requests
from bson.code import Code
from sqlalchemy import create_engine
import pymysql
import warnings
import mysql.connector
import re
import pandas as pd
import numpy as np
import collections as cl
from pandas.io.json import json_normalize

#### Establishing connetion with mongoDB

In [2]:
client = MongoClient("mongodb://localhost:27017/")

# Reads "reviews_electronics.16.json" and uploads each review as a separate document to the collection "reviews" in the database "amazon".

#### Creating a database called 'amazon'

In [3]:
amazon_db = client["amazon"]

#### Creating a collection called "electronics"

In [7]:
electronics_collection = amazon_db["electronics"]

#### Loading the data into a list of dictionaries

In [8]:
electronics_list = []
for line in open('reviews_electronics.16.json', 'r'):
    electronics_list.append(json.loads(line))

In [9]:
type(electronics_list)

list

#### Saving the data into mongoDB amazon database reviews collection

In [10]:
electronics_collection.insert_many(electronics_list)

#### Saving all the data from "electronics" collection in "amazon" databse of MongoDB into "electronics_stored".

"electronics_from_mongoDB" is a cursor and not where the data is being stored

In [11]:
electronics_from_mongoDB = amazon_db.electronics.find({})

In [14]:
electronics_from_mongoDB[1]

{'_id': ObjectId('5e5da775da1120978e1f8213'),
 'reviewerID': 'A2X8VX4DPMQFQQ',
 'asin': 'B00E4KP4W6',
 'reviewerName': 'lily68',
 'helpful': [1, 1],
 'reviewText': "I can't believe I waited to long to switch to a glass screen protector.  I love this.  It feels and looks like there is no protector on.  It does show fingerprints, which I think is inevitable unless you use a matte finish screen protector, but they wipe right away. I would definitely recommend this! Easier to apply than the films too!",
 'overall': 5.0,
 'summary': 'LOVE this screen protector!!',
 'unixReviewTime': 1393459200,
 'reviewTime': '02 27, 2014'}

#### Creating an empty list

In [13]:
electronics_stored = []

In [15]:
for x in electronics_from_mongoDB:
    electronics_stored.append(x)

#### Printing only the first 25 documents from the "electronics" collection

In [25]:
for index in range(0,25):
    print(electronics_stored[index])
    print('\n')

{'_id': ObjectId('5e5da775da1120978e1f8212'), 'reviewerID': 'AKM1MP6P0OYPR', 'asin': '0132793040', 'reviewerName': 'Vicki Gibson "momo4"', 'helpful': [1, 1], 'reviewText': 'Corey Barker does a great job of explaining Blend Modes in this DVD. All of the Kelby training videos are great but pricey to buy individually. If you really want bang for your buck just subscribe to Kelby Training online.', 'overall': 5.0, 'summary': 'Very thorough', 'unixReviewTime': 1365811200, 'reviewTime': '04 13, 2013'}


{'_id': ObjectId('5e5da775da1120978e1f8213'), 'reviewerID': 'A2X8VX4DPMQFQQ', 'asin': 'B00E4KP4W6', 'reviewerName': 'lily68', 'helpful': [1, 1], 'reviewText': "I can't believe I waited to long to switch to a glass screen protector.  I love this.  It feels and looks like there is no protector on.  It does show fingerprints, which I think is inevitable unless you use a matte finish screen protector, but they wipe right away. I would definitely recommend this! Easier to apply than the films too!

In [43]:
str(electronics_stored[0]["_id"])

'5e5da775da1120978e1f8212'

#### Converting the list of dictionaries into a dataframe

In [29]:
electronics_df = json_normalize(electronics_stored)

In [30]:
type(electronics_df)

pandas.core.frame.DataFrame

#### Converting columns to String type

In [70]:
electronics_df["_id"] = electronics_df._id.astype(str)

In [87]:
electronics_df["helpful"] = electronics_df.helpful.astype(str)

In [88]:
electronics_df.head()

,_id,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,5e5da775da1120978e1f8212,AKM1MP6P0OYPR,0132793040,"Vicki Gibson ""momo4""","[1, 1]",Corey Barker does a great job of explaining Bl...,5.0,Very thorough,1365811200,"04 13, 2013"
1,5e5da775da1120978e1f8213,A2X8VX4DPMQFQQ,B00E4KP4W6,lily68,"[1, 1]",I can't believe I waited to long to switch to ...,5.0,LOVE this screen protector!!,1393459200,"02 27, 2014"
2,5e5da775da1120978e1f8214,A1JEKX4D58576X,B00E4KP4W6,Mara Milagros,"[2, 2]","presentation of product is beautiful, everythi...",5.0,excellent,1394496000,"03 11, 2014"
3,5e5da775da1120978e1f8215,ATM6RRB90D8DV,B00E4KP4W6,"Matthew D. Irmen ""Jazz Fan""","[1, 2]",I bought this screen protector because I was u...,4.0,"Nice looking, easy to install glass screen pro...",1393891200,"03 4, 2014"
4,5e5da775da1120978e1f8216,A3QOYJ97A6OMX7,B00E4KP4W6,pharmdaddyd,"[1, 1]",this is a great screen protector for the price...,5.0,great for the price,1388361600,"12 30, 2013"


### Creating a connection to MySQL and creating a database called "amazon_SQL"

In [73]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd=""
)

mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE IF NOT EXISTS amazon_SQL")

mycursor.close()

mydb.close

<bound method MySQLConnection.close of <mysql.connector.connection.MySQLConnection object at 0x000001A561F80FC8>>

### Creating a connection to MySQL and creating a table called "electronics_SQL"

In [75]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database="amazon_SQL"
)

mycursor = mydb.cursor()

mycursor.execute("CREATE TABLE IF NOT EXISTS electronics_SQL (ID VARCHAR(50) PRIMARY KEY NOT NULL , reviewerID VARCHAR(50), asin VARCHAR(50), reviewerName VARCHAR(100), helpful VARCHAR(10), reviewText VARCHAR(2000), overall DOUBLE, summary VARCHAR(500), unixReviewTime BIGINT, reviewTime VARCHAR(30));")

mycursor.close()

mydb.close

<bound method MySQLConnection.close of <mysql.connector.connection.MySQLConnection object at 0x000001A5628829C8>>

In [82]:
engine = create_engine('mysql+pymysql://root:@localhost/amazon_SQL')

In [83]:
dbConnection = engine.connect()

In [89]:
electronics_df.to_sql('electronics_sql', dbConnection, index = False, if_exists='replace')

In [98]:
results = pd.read_sql("select * from electronics_sql", dbConnection);

In [99]:
pd.set_option('display.expand_frame_repr', False)
dbConnection.close()

### Printing the first 25 results

In [100]:
results.head(25)

,_id,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,5e5da775da1120978e1f8212,AKM1MP6P0OYPR,0132793040,"Vicki Gibson ""momo4""","[1, 1]",Corey Barker does a great job of explaining Bl...,5.0,Very thorough,1365811200,"04 13, 2013"
1,5e5da775da1120978e1f8213,A2X8VX4DPMQFQQ,B00E4KP4W6,lily68,"[1, 1]",I can't believe I waited to long to switch to ...,5.0,LOVE this screen protector!!,1393459200,"02 27, 2014"
2,5e5da775da1120978e1f8214,A1JEKX4D58576X,B00E4KP4W6,Mara Milagros,"[2, 2]","presentation of product is beautiful, everythi...",5.0,excellent,1394496000,"03 11, 2014"
3,5e5da775da1120978e1f8215,ATM6RRB90D8DV,B00E4KP4W6,"Matthew D. Irmen ""Jazz Fan""","[1, 2]",I bought this screen protector because I was u...,4.0,"Nice looking, easy to install glass screen pro...",1393891200,"03 4, 2014"
4,5e5da775da1120978e1f8216,A3QOYJ97A6OMX7,B00E4KP4W6,pharmdaddyd,"[1, 1]",this is a great screen protector for the price...,5.0,great for the price,1388361600,"12 30, 2013"
5,5e5da775da1120978e1f8217,AWO84NZ6VDYR2,B00E4KP4W6,shalane rogillio,"[0, 0]",I ordered two of these &#34;shatter proof&#34;...,1.0,"Not happy with this, would not recommend",1404777600,"07 8, 2014"
6,5e5da775da1120978e1f8218,A15ZEBTKX72NPB,B00E4KP4W6,"S. Marcus ""pdtpoet""","[1, 2]","For some reasons, the clear film screens that ...",5.0,Far better than film screens for iPhone or iPad!,1392249600,"02 13, 2014"
7,5e5da775da1120978e1f8219,A2ZI014DO0LWEW,B00E4KP4W6,SueinCa,"[0, 0]",This was just what the doctor ordered for my b...,5.0,Right On,1397174400,"04 11, 2014"
8,5e5da775da1120978e1f821a,A3IN9CE5GG8VKR,B00E4KP4W6,Tess Brown,"[0, 0]",This is the best screen protector. Very smoot...,5.0,Tempered Glass Screen Protector,1399593600,"05 9, 2014"
9,5e5da775da1120978e1f821b,A1U9ZR7HQ5EZ6L,B00E4KP4W6,Thinker623,"[1, 1]",Great glass cover for IPad. I had been using ...,5.0,Great glass cover for IPad.,1391126400,"01 31, 2014"
